In [11]:
#one proportion testing
#proportion test, 15 jellybeans 28 chocolate eggs
import pandas as pd
import scipy, scipy.stats
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
import statsmodels as sm
from statsmodels.stats.contingency_tables import mcnemar

In [2]:
#define counts, observations and proportion
#test will print statistic and p value
count = 15
nobs = 43
value = .5
stat, pval = proportions_ztest(count, nobs, value)
print(stat,pval)
#proportion of jellybeans is 38%

-2.079806538622099 0.03754328113448803


In [3]:
#now to test two proportions, two categories vs the whole
#we want to check the proportion of pink in the selection, 7 jelly, 12 chocolate
#now the first vector is the proportion of pink and the second is the proportion of candies
stat, pval = proportions_ztest([7, 12], [15, 28])
print(stat,pval)
#p value is not significant, meaning there is no real difference in the proportions 

0.23974366706563624 0.810528980523634


In [10]:
#goodness of fit chi square to compare sample to population, read that 90% of people like Star Wars, use your survey to check
SW= pd.read_csv("C:/Users/latri/Desktop/SW_survey_renamed.csv")
#wrangle like and dislike data
SW.FanYN.value_counts()
#create vectors for observed and expected, expected has to be whole numbers, add up all observed, multiply by expected proportion, subtract that number from the total exp for leftover percentage
observed_values = np.array([552, 284])
expected_values = np.array([752, 84])
scipy.stats.chisquare(observed_values, f_exp=expected_values)
#p value is super significant! which means if differs A LOT from the population (its not 90%)

Power_divergenceResult(statistic=529.3819655521784, pvalue=3.849512370977756e-117)

In [16]:
#mcnemar chi square, used to look at something over time that only has two data points
bakery= pd.read_csv("C:/Users/latri/Desktop/bakery_sales.csv")
#wondering if coffee sales are different between the beginning and end of the month 
#willl have to recode the data to make it usuable for analysis, check the structure

In [24]:
#separate out date column into string data
bakery1 = bakery['Date'].str.split('/', expand=True).rename(columns = lambda x: "Date" + str(x +1))
#and then put it back together
bakery3 = pd.concat([bakery, bakery1], axis=1)
#now we need to make date2 an integer, do a quick check and it shows up as a stringm so change it to integer!
bakery3.Date2 = bakery3.Date2.astype(int)
bakery3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21293 entries, 0 to 21292
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Date         21293 non-null  object
 1   Time         21293 non-null  object
 2   Transaction  21293 non-null  int64 
 3   Item         21293 non-null  object
 4   Date1        21293 non-null  object
 5   Date2        21293 non-null  int32 
 6   Date3        21293 non-null  object
dtypes: int32(1), int64(1), object(5)
memory usage: 1.1+ MB


In [25]:
#then we will recode the data to be beginning or end of month 
#use function for greater or less than 15 to change data
def month (series): 
    if series <= 15: 
        return 0
    if series > 16: 
        return 1
bakery3['DayR'] = bakery3["Date2"].apply(month)


In [26]:
#we will also recode data to be coffee or not coffee, since thats what we are after!
def item (series): 
    if series == "Coffee": 
        return 1
    if series != "Coffee": 
        return 0
bakery3['CoffeeYN'] = bakery3["Item"].apply(item)

In [27]:
#use crosstab to make contingency table since mcnemar doesnt take raw data
bakery_crosstab = pd.crosstab(bakery3['DayR'], bakery3['CoffeeYN'])
bakery_crosstab

CoffeeYN,0,1
DayR,,
0.0,8238,2841
1.0,7126,2491


In [28]:
#cant test for assumptions in python, so dont use this for important data!
result = sm.stats.contingency_tables.mcnemar(bakery_crosstab, exact=False, correction=True)
print(result)
#p is significant, sales do differ from the beginning and end of the month! but we will never know quite how in python

pvalue      0.0
statistic   1841.3420286946925
